In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle


In [2]:
# import training data
#dft_eng = pd.read_csv('../dataframes/dft_eng.csv')
#dft_fin = pd.read_csv('../dataframes/dft_fin.csv')
dft_jap = pd.read_csv('../dataframes/dft_jap.csv')

# import validation data
#dfv_eng = pd.read_csv('../dataframes/dfv_eng.csv')
#dfv_fin = pd.read_csv('../dataframes/dfv_fin.csv')
dfv_jap = pd.read_csv('../dataframes/dfv_jap.csv')

## Bag of Words

### Functions to create BoW vectors

In [3]:
def make_bow_vector(sentence, countt):
    vec = np.zeros(len(countt))
    for word in sentence:
        vec[countt[word]] += 1
    return vec

def ans_freq(que, doc):

    freq = [x for x in que if x in doc]
    freq = len(freq)/len(que)
    return freq

def freq_words_in_text (df_t,df_v):
    for df in [df_t,df_v]:
        frequency = []
        for question, answer in zip(df['question_text_tokenized'], df['document_plaintext_tokenized']):
            frequency.append(ans_freq(eval(question), eval(answer)))

        df['word_frequency_score'] = frequency

### Combine and flatten

In [4]:
# state which language to use
dft = dft_jap
dfv = dfv_jap


In [5]:
t_comb2 = [str(que)+ " "+ str(doc) for que, doc in zip(dft["question_text"],dft["document_plaintext"])]
v_comb2 = [str(que)+ " "+ str(doc) for que, doc in zip(dfv["question_text"],dfv["document_plaintext"])]

In [6]:
t_comb2.extend(v_comb2)

In [7]:
CountVec = CountVectorizer(ngram_range=(1,1))
Count_data = CountVec.fit_transform(t_comb2)

In [8]:
freq_words_in_text(dft,dfv)

In [9]:
ct = Count_data.toarray()[:len(dft)]
cv = Count_data.toarray()[len(dft):]

In [10]:
ct = [l.tolist() for l in ct]

In [11]:
cv = [l.tolist() for l in cv]

In [16]:
xt = []
for index in range(len(dft)):
    xt.append(np.append(dft['word_frequency_score'][index],dft['bow6'][index]))
    
xv = []
for index in range(len(dfv)):
    xv.append(np.append(dfv['word_frequency_score'][index],dfv['bow6'][index]))

In [17]:
X_train = np.array(xt)
X_val = np.array(xv)
y_train = dft.label.values
y_val = dfv.label.values

In [18]:
clf = LogisticRegression(C=1000, penalty='l1', random_state=1, solver='liblinear').fit(X_train, y_train)

In [19]:
val_pred = clf.predict(X_val)
accuracy_score(val_pred, y_val)

0.6303088803088803

In [20]:
train_pred = clf.predict(X_train)
accuracy_score(train_pred, y_train)

0.9995443153337891

In [21]:
filename = '../models/logres_model_jap.sav'
pickle.dump(clf, open(filename, 'wb'))